# Краулер fedstat.ru

По адресу [https://fedstat.ru/opendata/](https://fedstat.ru/opendata/) доступна страница с пагинацией, однако все ссылки
на ресурсы находятся в HTML файле. Таким образом сбор данных происходит так:
* Качаем URL https://fedstat.ru/opendata/, парсим адреса всех страниц конкретных наборов данных
* Качаем отдельные страницы, парсим метаданные, собираем из них pandas DataFrame. Данные сравнительно легко парсятся, так как используются микроформаты
* Качаем непосретсвенно открытые данные

Закачка происходит через прокси - сайт весьма стабильно работает через Tor

In [129]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm
import os

In [4]:
# socks прокси Tor'а
proxies = {
  'http': 'socks5://localhost:9050',
  'https': 'socks5://localhost:9050'
}
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'
}

In [10]:
res = requests.post("https://fedstat.ru/opendata/", proxies=proxies, headers=headers)
doc = BeautifulSoup(res.text)

In [23]:
links = list(map(lambda x: x.attrs['href'], doc.select("[typeof='foaf:Document'] a")))
print(len(links), len(set(links)))

15240 7620


In [83]:
# для возобновления обработки в случае падения
processed = set()
data = []

In [87]:
# Собираем информацию о всех доступных наборах данных
for link in tqdm(set(links)):
    if link in processed:
        continue
    url = "https://fedstat.ru/%s" % link
    res = requests.post(url, proxies=proxies, headers=headers)
    doc = BeautifulSoup(res.text)
    obj = {'url': url}
    for about in [link, '#publisher', '#data']:
        for node in doc.select("[typeof='sioc:Item foaf:Document'] [about='%s'] [property]" % about):
            value = node.attrs.get('content', node.text)
            if node.attrs['property'] in obj and value != obj[node.attrs['property']]:
                raise Exception("Conflict: %s, %s" % (node, obj))
            obj[node.attrs['property']] = value
            
    if 'dc:source' not in obj:
        print(url)
        continue
    data.append(obj)
    processed.add(link)

https://fedstat.ru/#



In [ ]:
!mkdir -p ../../data/fedstat.ru/

In [89]:
# Сохраняем метаданные
df = pd.DataFrame(data).drop_duplicates()
print(df.shape, df.url.nunique(), df['dc:source'].nunique(), df['dc:identifier'].nunique())
df.head()

(7619, 14) 7619 7619 7619


,url,dc:identifier,dc:title,dc:description,dc:creator,dc:created,dc:valid,dc:subject,foaf:name,foaf:phone,foaf:mbox,dc:source,dc:format,dc:provenance
0,https://fedstat.ru//opendata/7708234640-fiveaf...,7708234640-fiveafourathreeanineatwo,Количество зарегистрированных в собственности ...,Отражается общее количество зарегистрированных...,Федеральная служба государственной регистрации...,31.05.2019,28.09.2021,,Базылева Е.В.,(495) 531-08-00 доб. 14-48,Bazileva_EV@rosreestr.ru,http://fedstat.ru/opendata/7708234640-fiveafou...,XML,Актуализация данных показателя на дату 31.05.2019
1,https://fedstat.ru//opendata/7707418081-fivean...,7707418081-fiveanineanineatwoafour,Доля обучающихся по общеобразовательным програ...,Приказ Минпросвещения России от 17 апреля 2019...,Министерство просвещения Российской Федерации,13.04.2020,30.06.2021,,Бондарь Евгений Витальевич,8 (495) 587-01-10 (3221),bondar-ev@edu.gov.ru,http://fedstat.ru/opendata/7707418081-fiveanin...,XML,Актуализация данных показателя на дату 13.04.2020
2,https://fedstat.ru//opendata/7708234640-fiveas...,7708234640-fiveasevenafourafiveafive,Среднесписочная численность работников в орган...,Среднесписочная численность работников организ...,Федеральная служба государственной статистики,26.08.2020,09.12.2021,,Тишкина О.В.,(495) 607-41-41 (доб. 99227),TishkinaOV@gks.ru,http://fedstat.ru/opendata/7708234640-fiveasev...,XML,Актуализация данных показателя на дату 26.08.2020
3,https://fedstat.ru//opendata/7708234640-threea...,7708234640-threeafouraoneatwoatwo,Доходы местного бюджета на текущий год,"Показатели местного бюджета, утвержденного на ...",Федеральная служба государственной статистики,03.11.2020,14.06.2018,\nПоказатель\n,Волкова И.В.,(495)607-41-41 (доб. 99309),Volkova_i@gks.ru,http://fedstat.ru/opendata/7708234640-threeafo...,XML,Актуализация данных показателя на дату 03.11.2020
4,https://fedstat.ru//opendata/7708234640-fiveae...,7708234640-fiveaeightatwoazeroasix,Численность основного персонала в учреждениях ...,Данные заполняются из статистической формы № 7-НК,Министерство культуры Российской Федерации,05.10.2018,25.05.2021,,Коновалова Наталья Александровна,8 (495) 2616525,kna@mkrf.ru,http://fedstat.ru/opendata/7708234640-fiveaeig...,XML,Актуализация данных показателя на дату 05.10.2018


In [116]:
# формат всех данных - XML
df['dc:format'].value_counts()

XML    7619
Name: dc:format, dtype: int64

In [91]:
!mkdir -p ../../data/fedstat.ru/data/

In [124]:
# Часть данных недоступно - открывается HTML документ с ошибкой
for _, row in tqdm(df.iterrows()):
    dataset_id = row['dc:identifier']
    if dataset_id in processed:
        continue
    res = requests.post(row['dc:source'], proxies=proxies, headers=headers)
    assert res.status_code == 200, "%s: %s" % (dataset_id, res)
    content = res.content
    if '<p>Нет данных для выбранного сочетания признаков.</p>' in content.decode("utf-8"):
        print("No data for %s" % dataset_id)
        processed.add(dataset_id)
        continue
    if '<!DOCTYPE html>' in content.decode("utf-8"):
        print("Non xml data in %s" % dataset_id)
        processed.add(dataset_id)
        continue

    with open("../../data/fedstat.ru/data/%s_%s" % (dataset_id, row['dc:source'].split("/")[-1]), "wb") as out:
        out.write(content)
    processed.add(dataset_id)

Non xml data in 7708234640-sixazeroanineanineaseven
Non xml data in 7708234640-sixazeroazeroazeroaeight
Non xml data in 7708234640-sixaoneathreeafouranine
Non xml data in 7708234640-sixaoneathreeathreeathree
Non xml data in 7708234640-sixazeroaoneathreeaseven
Non xml data in 7708234640-fiveasevenanineathreeatwo
Non xml data in 7708234640-sixaoneatwoaeightathree
No data for 7708234640-threeaeightaoneasevenasix
Non xml data in 7708234640-sixazeroaeightatwoaseven
No data for 7708234640-fiveaeightaoneaoneaeight
No data for 7708234640-fiveaoneasixatwoafour
No data for 7708234640-threeafouraoneasevenafive
No data for 7708234640-threeathreeasevenathreeafour
No data for 7708234640-fourathreeasixasixatwo
No data for 7708234640-fiveaeightazeroaoneafour
No data for 7708234640-fouratwoanineafouraseven
No data for 7708234640-fouratwoazeroanineaeight
Non xml data in 7708234640-sixaoneatwoasevenanine
No data for 7708234640-threeaoneazeroafouratwo
No data for 7708234640-threeasixasevenaoneaseven
No da

No data for 7708234640-fourazeroafiveafourafour
Non xml data in 7708234640-sixazeroasevenatwoasix
No data for 7708234640-fiveaeightazeroaoneafive
No data for 7708234640-fiveaoneasixathreeafour
No data for 7708234640-fourazeroasixafourafour
Non xml data in 7708234640-fiveanineanineasevenafour
No data for 7708234640-threeathreeasevenaoneazero
Non xml data in 7708234640-sixaoneazeroatwoazero
Non xml data in 7708234640-fouratwoasixafouraseven
No data for 7708234640-fourathreeasixafiveafour
Non xml data in 7708234640-sixazeroatwoatwoazero
Non xml data in 7708234640-fiveanineasevenazeroazero
Non xml data in 7708234640-sixaoneazeroazeroaseven
No data for 7708234640-fourathreeasixafourazero
No data for 7708234640-fourathreeasixathreeafour
No data for 7708234640-fourazeroafiveazeroasix
No data for 7708234640-fouratwoazeroanineanine
Non xml data in 7707418081-sixazeroanineanineaone
Non xml data in 7710539135-fiveanineanineafouraeight
No data for 7708234640-fourazeroasixatwoatwo
No data for 77082

No data for 7708234640-fouratwoanineafouranine
Non xml data in 7708234640-fiveaeightafourafourafive
No data for 7708234640-fouratwoanineafourasix
Non xml data in 7708234640-sixazeroaeightafourazero
Non xml data in 7708234640-sixaoneafouratwoathree
Non xml data in 7708234640-sixaoneafourasevenazero
Non xml data in 7708234640-sixaoneafourathreeafive
No data for 7708234640-threeathreeasixaoneaeight
No data for 7708234640-threeasevenafiveafourafour
No data for 7708234640-fiveaeightathreeazeroanine
No data for 7708234640-fiveazeroatwoathreeazero
Non xml data in 7708234640-sixazeroatwoazeroafour
Non xml data in 7708234640-sixaoneazeroatwoaone
No data for 7708234640-fiveasevenaeightatwoafive
No data for 7708234640-threeafiveanineaeightasix
Non xml data in 7708234640-sixazeroaeightatwoanine
Non xml data in 7708234640-sixaoneathreeatwoaone
No data for 7708234640-fourathreeasixasixafour
No data for 7708234640-fourathreeasixasixafive
No data for 7708234640-fourathreeasixafiveaseven
Non xml data i

Non xml data in 7708234640-fiveaeightatwoathreeazero
Non xml data in 7707418081-sixazeroasevenaeightathree
Non xml data in 7708234640-fiveaeightanineatwoaseven
Non xml data in 7708234640-sixazeroanineanineasix
No data for 7708234640-fourathreeazeroatwoaeight
No data for 7708234640-threeaeightaoneafouranine
Non xml data in 7708234640-fiveafiveaeightanineafour
Non xml data in 7708234640-sixazeroatwoatwoasix
Non xml data in 7702235133-threeasevenanineasixasix
No data for 7708234640-fourazeroafouraeightazero
No data for 7708234640-fourazeroasixatwoathree
Non xml data in 7708234640-fiveasixanineasevenanine
No data for 7708234640-fiveaeightasixasixatwo
Non xml data in 7708234640-sixazeroasevenasevenaone
Non xml data in 7708234640-sixatwoazeroatwoaone
No data for 7708234640-threeasixasixatwoaone
No data for 7708234640-threeasixazeroazeroasix
Non xml data in 7708234640-sixaoneathreeafourathree
No data for 7708234640-fourafiveafourasevenasix
Non xml data in 7708234640-fiveasixanineaeightafour
N

In [131]:
# Добавляем имя файла в список данных и проверяем что файл был скачан
def get_filename(row):
    filename = "%s_%s" % (row['dc:identifier'], row['dc:source'].split("/")[-1])
    if os.path.exists("../../data/fedstat.ru/data/%s" % filename):
        return filename
    else:
        return "missing"
df['filename'] = df.apply(get_filename, axis=1)
df.to_csv("data/metadata_20220410.csv", index=False)
df.sample(n=10)

,url,dc:identifier,dc:title,dc:description,dc:creator,dc:created,dc:valid,dc:subject,foaf:name,foaf:phone,foaf:mbox,dc:source,dc:format,dc:provenance,filename
4215,https://fedstat.ru//opendata/7708234640-fiveao...,7708234640-fiveaoneasixaoneaseven,Оплата сторонних организаций по предоставлению...,Общий объем израсходованных денежных средств.,Федеральная служба государственной статистики,15.12.2020,03.05.2018,,Казинская М.Н.,(495) 607-41-41 (доб. 99368),ca_KazinskayaMN@gks.ru,http://fedstat.ru/opendata/7708234640-fiveaone...,XML,Актуализация данных показателя на дату 15.12.2020,7708234640-fiveaoneasixaoneaseven_data-2020-12...
1031,https://fedstat.ru//opendata/7708234640-fiveas...,7708234640-fiveasevenafouranineaone,"Число многоквартирных домов, в которых собстве...",Требуется заполнить,Министерство строительства и жилищно-коммуналь...,26.04.2018,03.11.2021,,Смирнов И.В.,8 (495) 647-15-80 доп. 53044,Ivan.Smirnov@minstroyrf.ru,http://fedstat.ru/opendata/7708234640-fiveasev...,XML,Актуализация данных показателя на дату 26.04.2018,7708234640-fiveasevenafouranineaone_data-2018-...
1598,https://fedstat.ru//opendata/7708234640-threea...,7708234640-threeaoneathreeasevenazero,Производство яиц,"Количество яиц, полученных в хозяйстве за отч...",Федеральная служба государственной статистики,22.09.2020,04.04.2022,,Лавреев Ю.Б.,(495) 607-41-41 (доб. 99832),lavreev@gks.ru,http://fedstat.ru/opendata/7708234640-threeaon...,XML,Актуализация данных показателя на дату 22.09.2020,7708234640-threeaoneathreeasevenazero_data-202...
393,https://fedstat.ru//opendata/7708234640-threea...,7708234640-threeathreeanineathreeatwo,Проведено торгов (лотов) и запросов котировок ...,Показатель формируется по главным распорядител...,Федеральная служба государственной статистики,25.02.2019,11.04.2014,,Зотова Т.Л.,"(495) 607-41-70, 99348",ZotovaTL@gks.ru,http://fedstat.ru/opendata/7708234640-threeath...,XML,Актуализация данных показателя на дату 25.02.2019,7708234640-threeathreeanineathreeatwo_data-201...
1571,https://fedstat.ru//opendata/7708234640-threea...,7708234640-threeasevenasixazeroathree,"Численность работников, замещавших государстве...","Фактическая численность работников, замещавших...",Федеральная служба государственной статистики,27.08.2020,07.04.2020,,Новосельцева Е.А.,(495) 607-41-41 (доб. 99255),NovoseltsevaEA@gks.ru,http://fedstat.ru/opendata/7708234640-threease...,XML,Актуализация данных показателя на дату 27.08.2020,7708234640-threeasevenasixazeroathree_data-202...
7026,https://fedstat.ru//opendata/7708234640-threea...,7708234640-threeasixaoneatwoazero,Передано и отгружено драгоценных металлов в сч...,Значения показателей формируются на основании ...,Министерство финансов Российской Федерации,12.04.2017,30.12.2020,,Лобзов И.К.,646-40-17,,http://fedstat.ru/opendata/7708234640-threeasi...,XML,Актуализация данных показателя на дату 12.04.2017,7708234640-threeasixaoneatwoazero_data-2017-04...
3493,https://fedstat.ru//opendata/7708234640-fourao...,7708234640-fouraoneazeroazeroasix,Импорт Российской Федерации по группам стран,Импорт товаров - ввоз товаров на таможенную те...,Федеральная таможенная служба,17.02.2022,28.12.2021,,Гаврилин А.Д.,449-73-07,gavrilinAD@ca.customs.ru,http://fedstat.ru/opendata/7708234640-fouraone...,XML,Актуализация данных показателя на дату 17.02.2022,7708234640-fouraoneazeroazeroasix_data-2022-02...
1646,https://fedstat.ru//opendata/7708234640-fiveas...,7708234640-fiveasevenathreeafourazero,"Доля населения, проживающего в аварийном и вет...",Рассчитывается как отношение численности насел...,Федеральная служба государственной статистики,26.06.2019,09.11.2017,,Горнов Г.С.,"(495) 607-49-81, 99735",ca_GornovGS@gks.ru,http://fedstat.ru/opendata/7708234640-fiveasev...,XML,Актуализация данных показателя на дату 26.06.2019,7708234640-fiveasevenathreeafourazero_data-201...
1238,https://fedstat.ru//opendata/7708234640-fivean...,7708234640-fiveanineaoneatwoaeight,Количество привлеченных к участию в реализации...,Расчет показателя произ